## Caricamento del modello

In [ ]:
from torchvision.models import alexnet, AlexNet_Weights
import torch.nn as nn
import torch
import torch.optim as optim

# Caricamento delle informazioni salvate
loaded_info = torch.load('model/modello_fia12.pth')

# Caricamento di AlexNet senza i pesi preaddestrati
alexnet = alexnet(weights=None)

# Modifica del primo strato convoluzionale
alexnet.features[0] = nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2)

# Modifica dell'ultimo strato fully-connected per la classificazione binaria
alexnet.classifier[6] = nn.Linear(4096, 1)

# Aggiunta della funzione di attivazione sigmoide all'ultimo strato
alexnet.classifier.add_module("7", nn.Sigmoid())

# Congelamento dei pesi dei primi strati
for param in alexnet.features.parameters():
    param.requires_grad = False

# Caricamento stato modello
alexnet.load_state_dict(loaded_info['model_state_dict'])

# Ricreazione dello stato dell'ottimizzatore
optimizer = optim.Adam(alexnet.parameters(), lr=0.001)
optimizer.load_state_dict(loaded_info['optimizer_state_dict'])

# Ricreazione del criterio di perdita
criterion = nn.BCELoss()  # o qualsiasi altro criterio usato
criterion.load_state_dict(loaded_info['criterion_state_dict'])

# Recupero delle altre informazioni
epoch = loaded_info['epoch']
preprocessing_params = loaded_info['preprocessing']

# Metti il modello in modalità di valutazione
alexnet.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
from torchvision import transforms
from PIL import Image

# Definisci le trasformazioni
standard_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Ridimensionamento per AlexNet
    transforms.Grayscale(num_output_channels=1),  # Conversione in scala di grigi
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229])  # Normalizzazione per un singolo canale
])

# Crea un dizionario per mappare gli indici alle etichette
class_labels = {0: "Non demented", 1: "Demented"}

## Valutazione

In [ ]:
# Carica l'immagine singola e applica le trasformazioni
img = Image.open("test/32 (50).jpg")
img_t = standard_transforms(img)
batch_t = torch.unsqueeze(img_t, 0)

# Esegui la predizione
with torch.no_grad():
    output = alexnet(batch_t)

# output è ora un tensore contenente le probabilità delle classi

predicted_probability = output.item()
print(predicted_probability)

if predicted_probability > 0.5:
    predicted_label = "Non demented"
else:
    predicted_label = "Demented"

print(f"La predizione del modello è: '{predicted_label}'")

In [ ]:
import os

# Percorso della cartella con le immagini di test
test_folder = "test"

# Contatori per le predizioni
count_non_demented = 0
count_demented = 0

# Lista per salvare i risultati delle predizioni
predictions = []

# Itera su tutti i file nella cartella di test
for img_file in os.listdir(test_folder):
    img_path = os.path.join(test_folder, img_file)

    # Assicurati che il file sia un'immagine
    if img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Carica l'immagine e applica le trasformazioni
        img = Image.open(img_path)
        img_t = standard_transforms(img)
        batch_t = torch.unsqueeze(img_t, 0)

        # Esegui la predizione
        with torch.no_grad():
            output = alexnet(batch_t)

        predicted_probability = output.item()

        if predicted_probability > 0.5:
            pred = "Non demented"
            count_non_demented += 1
        else:
            pred = "Demented"
            count_demented += 1

        # Salva la predizione
        predictions.append((img_file, pred))

# Stampa i risultati
print(f"Numero di immagini classificate come 'Demented': {count_demented}")
print(f"Numero di immagini classificate come 'Non demented': {count_non_demented}")

Numero di immagini classificate come 'Demented': 229
Numero di immagini classificate come 'Non demented': 411
